In [1]:
import os
import glob
import soundfile as sf
import librosa
import random 
import numpy as np

DATA_PATH = '/data3/VCTK_spec/VCTK-Corpus/wav48'
TXT_PATH = '/data3/VCTK/VCTK-Corpus/txt'
COMMON_TXT_PATH = '/data3/VCTK/VCTK-Corpus/txt'
ALL_DATA_PATH = sorted(glob.glob(os.path.join(DATA_PATH, '*/*.npy')))
ALL_TXT_PATH = sorted(glob.glob(os.path.join(TXT_PATH, '*/*.txt')))
SR = 16000
CUT = 12

print('length of data', len(ALL_DATA_PATH))
print('length of txt data', len(ALL_TXT_PATH))

length of data 44257
length of txt data 44085


## Add files to the train list

In [2]:
def read_text(path):
    with open(path, 'r') as f:
        line = f.readline()
        if '\n' in line:
            return line
        else:
            line = line + '\n'
            return line

    
def check_audio_length(path):
    audio, _ = sf.read(path)
    if len(audio) < (CUT * SR):
        return True
    else:
        return False

def check_np_length(path):
    data = np.load(path)
    _, length = data.shape
    
    if int(length * 200) < (CUT * SR):
        return True
    else:
        return False
    
    
def random_sample(used_path, all_path):
    all_path.remove(used_path)
    path = random.sample(all_path, 1)[0]
    while True:
        if check_np_length(path) == False:
            path = random.sample(all_path, 1)[0]
        else:
            return path
        
def check_same(data_path, txt_path):
    print(data_path)
    data_id = data_path.split('/')[-1].split('.')[0]
    print(data_id)
    txt_id = txt_path.split('/')[-1].split('.')[0]
    print(txt_id)
    if data_id == txt_id:
        return True
    else:
        return False
    
def convert_to_txt_path(data_path):
    
    data_id = data_path.split('/')[-1].split('.')[0]
    p_id = data_id.split('_')[0]
    txt_path = os.path.join(COMMON_TXT_PATH, p_id, data_id+'.txt')
    return txt_path    
    
    

In [3]:
all_pid = [ x.split('/')[-1] for x in glob.glob(COMMON_TXT_PATH + '/*')]
tr_pid = all_pid[:85]
ts_pid = all_pid[85:]
print('tr_pid', tr_pid)
print('ts pid', ts_pid)

tr_pid ['p264', 'p275', 'p336', 'p252', 'p304', 'p316', 'p339', 'p360', 'p265', 'p330', 'p293', 'p285', 'p305', 'p226', 'p236', 'p269', 'p281', 'p283', 'p351', 'p276', 'p376', 'p312', 'p287', 'p335', 'p261', 'p298', 'p254', 'p310', 'p306', 'p237', 'p240', 'p239', 'p300', 'p241', 'p247', 'p256', 'p229', 'p255', 'p334', 'p271', 'p263', 'p279', 'p243', 'p333', 'p288', 'p253', 'p244', 'p248', 'p329', 'p238', 'p301', 'p272', 'p374', 'p277', 'p361', 'p323', 'p234', 'p246', 'p347', 'p260', 'p231', 'p295', 'p262', 'p273', 'p308', 'p228', 'p250', 'p259', 'p307', 'p232', 'p314', 'p294', 'p317', 'p284', 'p266', 'p267', 'p292', 'p362', 'p230', 'p245', 'p345', 'p343', 'p225', 'p233', 'p258']
ts pid ['p364', 'p251', 'p280', 'p341', 'p270', 'p326', 'p303', 'p311', 'p257', 'p363', 'p249', 'p297', 'p313', 'p282', 'p278', 'p302', 'p299', 'p268', 'p318', 'p286', 'p227', 'p274', 'p340']


In [4]:
train_path = '/home/xliu0/workspace/tacotron2/filelists/train_filelist_vctk.txt'
test_path = '/home/xliu0/workspace/tacotron2/filelists/test_filelist_vctk.txt'

train_path_alt = '/home/xliu0/workspace/tacotron2/filelists/train_filelist_vctk_alt.txt'
test_path_alt = '/home/xliu0/workspace/tacotron2/filelists/test_filelist_vctk_alt.txt'


for data_path in ALL_DATA_PATH:
    if data_path.split('/')[-2] in tr_pid:
        if check_np_length(data_path) == False:
            continue
        txt_path = convert_to_txt_path(data_path)
        if os.path.exists(txt_path) == False:
            continue
        else:
            with open(train_path, 'a') as f1, open(train_path_alt, 'a') as f2:
            # Write into train file:
                s1 = data_path + '|' + read_text(txt_path)
                f1.write(s1)
                # Write into alt file 
                data_folder_path = data_path.split('/')[:-1]
                data_folder_path = '/'.join(data_folder_path)
                all_wav_data_folder = sorted(glob.glob(data_folder_path + '/*.npy'))
                alt_wav_path = random_sample(data_path, all_wav_data_folder)
                alt_txt_path = convert_to_txt_path(alt_wav_path)
                s2 = alt_wav_path + '|' + read_text(alt_txt_path)
                f2.write(s2)
    elif data_path.split('/')[-2] in ts_pid:
        if check_np_length(data_path) == False:
            continue
        txt_path = convert_to_txt_path(data_path)
        if os.path.exists(txt_path) == False:
            continue
        else:
            with open(test_path, 'a') as f1, open(test_path_alt, 'a') as f2:
            # Write into train file:
                s1 = data_path + '|' + read_text(txt_path)
                f1.write(s1)
                # Write into alt file 
                data_folder_path = data_path.split('/')[:-1]
                data_folder_path = '/'.join(data_folder_path)
                all_wav_data_folder = sorted(glob.glob(data_folder_path + '/*.npy'))
                alt_wav_path = random_sample(data_path, all_wav_data_folder)
                alt_txt_path = convert_to_txt_path(alt_wav_path)
                s2 = alt_wav_path + '|' + read_text(alt_txt_path)
                f2.write(s2)
    else:
        continue


